# FEV vs. Alpha Analysis

For each VGG19 layer, this notebook plots **mean FEV** vs. **alpha** (where alpha is the slope of the log–log PCA variance spectrum).

- **Mean FEV** is taken from the regression: VGG19 features (reduced to N PCs) → Neural data (original dimensionality)

- For each layer, we select the **best FEV** (i.e., highest across PC counts)

In [1]:
from cortexlib.power_law import PowerLawAlphaEstimator
from cortexlib.mouse import CortexlabMouse
from cortexlib.utils import file as futils
from cortexlib.utils.logging import Logger
from cortexlib.utils.random import set_global_seed, GLOBAL_SEED
from cortexlib.utils.plotting import vgg19_colours
from sklearn.decomposition import PCA
from adjustText import adjust_text
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import torch

logger = Logger()
set_global_seed()

In [2]:
MOUSE_ID = futils.get_mouse_id()
logger.info(f"This notebook is running for mouse {MOUSE_ID}")

16:51:46 | INFO     | ℹ️ This notebook is running for mouse m01_d2


In [3]:
neural_plae = PowerLawAlphaEstimator()
mouse = CortexlabMouse(mouse_id=MOUSE_ID)

logger.progress(f"Computing null distributions for all neurons in mouse {mouse.id}")
null_srv_all_neurons = mouse.compute_null_all_neurons(n_shuffles=100)
logger.success(f"Null distributions computed")

real_srv_all_neurons = mouse.compute_real_srv_all_neurons()
reliable_neuron_indices = mouse.get_reliable_neuron_indices(
            null_srv_all_neurons, real_srv_all_neurons, percentile_threshold=99)
neural_responses_mean, neural_responses, _ = mouse.get_responses_for_reliable_neurons(reliable_neuron_indices, real_srv_all_neurons, num_neurons=500)

logger.info(f"Neural responses shape: {neural_responses.shape}")

neural_alpha = neural_plae.compute_alpha(neural_responses_mean)

16:51:46 | INFO     | ⏳ Computing null distributions for all neurons in mouse m01_d2...
16:52:12 | SUCCESS  | ✅ Null distributions computed!
16:52:12 | INFO     | ℹ️ Neural responses shape: (1380, 2, 500)


In [4]:
fev_results = futils.read_json_file_as_dataframe('./prediction.json')
best_fev_per_layer = fev_results.loc[fev_results.groupby("layer")["mean_fev"].idxmax()]

best_fev_per_layer

,layer,n_pcs,test_r2,mean_fev
0,conv2_2,NaN,0.021229,0.128841
5,conv3_4,NaN,0.017866,0.108915
10,conv4_4,NaN,0.018915,0.128420
15,conv5_4,NaN,0.012395,0.099506
20,fc2,NaN,0.009707,0.095504


In [5]:
vgg19_data = futils.load_model_features(futils.Model.VGG19, MOUSE_ID)
vgg19_feats = vgg19_data['features']

16:52:12 | INFO     | ℹ️ Loading model features from /Users/callummessiter/workspace/msc-neuro/research-project/analysis/mouse_m01_d2/_model_features/vgg19_features_mouse_m01_d2.pt


In [7]:
# Save explained variance info

num_components = 128
plot_components = 128
thresholds = [0.80, 0.90]

raw_features = vgg19_feats
layer_names = list(raw_features.keys())
rand_feat_indices = torch.randperm(num_components)[:num_components]

def run_pca(data, n_components=num_components):
    pca = PCA(n_components=n_components, random_state=GLOBAL_SEED)
    pca.fit(data)
    return np.cumsum(pca.explained_variance_ratio_), pca.explained_variance_ratio_

def num_components_for_variance(cumulative_ev, threshold):
    return np.argmax(cumulative_ev >= threshold) + 1

def subsample_features(feats, indices):
    if feats.ndim > 2:
        feats = feats.reshape(feats.shape[0], -1)
    return feats[:, indices] if feats.shape[1] >= len(indices) else feats

features = {
    name: subsample_features(raw_features[name], rand_feat_indices) for name in layer_names
}
features['neural'] = neural_responses_mean[:, rand_feat_indices]

cumulative_ev = {}
ev = {}
for name, feats in features.items():
    cumulative_ev[name], ev[name] = run_pca(feats)

components_needed = {
    t: {name: num_components_for_variance(cumulative_ev[name], t) for name in features}
    for t in thresholds
}

json_data = [
    {
        "mouse_id": MOUSE_ID,
        "model": futils.get_model(),
        "layer": layer,
        "num_principal_components_80pc_ev": int(components_needed[0.80][layer]),
        "num_principal_components_90pc_ev": int(components_needed[0.90][layer]),
        "cumulative_ev": cumulative_ev[layer].tolist(),
    }
    for layer in layer_names
]

futils.write_json_file(json_data, './explained_variance.json')

In [ ]:
power_law_alpha_estimator = PowerLawAlphaEstimator()

alpha_results = []

logger.progress(f"Computing α for VGG19 features across layers")
for layer, feats in tqdm(vgg19_feats.items(), desc="α for all VGG19 layers"):
    logger.progress(f"VGG19 {layer}")
    
    feats = feats if feats.dim() <= 2 else feats.view(feats.size(0), -1)
    alpha = power_law_alpha_estimator.compute_alpha(feats)

    alpha_results.append({
        'layer': layer,
        'alpha': alpha['alpha'],
        'alpha_no_pc1': alpha['alpha_no_pc1'],
    })

logger.success(f"Alphas computed")

In [ ]:
futils.write_json_file(alpha_results, './dimensionality.json')

In [ ]:
alpha_results_df = pd.DataFrame(alpha_results)
alpha_results_df

In [ ]:
merged_results = pd.merge(best_fev_per_layer, alpha_results_df, on='layer')
merged_results

In [ ]:
colours = merged_results['layer'].map(vgg19_colours)

plt.figure(figsize=(10, 6))
plt.scatter(merged_results['alpha'], merged_results['mean_fev'], c=colours)

texts = []
for _, row in merged_results.iterrows():
    texts.append(
        plt.text(row['alpha'], row['mean_fev'], row['layer'],
                 color=vgg19_colours[row['layer']], fontsize=10)
    )

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray'))

plt.xlabel('Alpha (Power Law Exponent)')
plt.ylabel('Mean FEV')
plt.title('VGG19 Layers: Alpha vs. Predictive Power (Neural Data)')
plt.grid(True)
plt.tight_layout()
plt.show()